In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle


In [3]:
## Import data 
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Remove irrelevent variables
data.drop(["RowNumber","CustomerId","Surname"], axis=1, inplace=True)

In [ ]:
# Gender Encoding
le_gen=LabelEncoder()
data["Gender"]=le_gen.fit_transform(data["Gender"])

In [11]:
# Geography encoding
ohe_geo=OneHotEncoder()
encoded_geo=ohe_geo.fit_transform(data[["Geography"]]).toarray()
encoded_geo

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
# Merging Data
geo_encoded_df=pd.DataFrame(encoded_geo,columns=ohe_geo.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# Scaling
X=data.drop("Exited",axis=1)
y=data["Exited"]
scaler=StandardScaler()
X=scaler.fit_transform(X)


array([[-0.32622142, -1.09598752,  0.29351742, ...,  0.99720391,
        -0.57873591, -0.57380915],
       [-0.44003595, -1.09598752,  0.19816383, ..., -1.00280393,
        -0.57873591,  1.74273971],
       [-1.53679418, -1.09598752,  0.29351742, ...,  0.99720391,
        -0.57873591, -0.57380915],
       ...,
       [ 0.60498839, -1.09598752, -0.27860412, ...,  0.99720391,
        -0.57873591, -0.57380915],
       [ 1.25683526,  0.91241915,  0.29351742, ..., -1.00280393,
         1.72790383, -0.57380915],
       [ 1.46377078, -1.09598752, -1.04143285, ...,  0.99720391,
        -0.57873591, -0.57380915]])

In [18]:
## Train test split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=17)

In [21]:
## Save the encoders and sscaler
with open('le_gen.pkl','wb') as file:
    pickle.dump(le_gen,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


### ANN Implemation


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [29]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
]

)

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [32]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [33]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [34]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [35]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 0.3932 - accuracy: 0.8397 - val_loss: 0.3780 - val_accuracy: 0.8400
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3508 - accuracy: 0.8572 - val_loss: 0.3714 - val_accuracy: 0.8440
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3469 - accuracy: 0.8560 - val_loss: 0.3592 - val_accuracy: 0.8525
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8581 - val_loss: 0.3676 - val_accuracy: 0.8360
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3359 - accuracy: 0.8590 - val_loss: 0.3584 - val_accuracy: 0.8505
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3354 - accuracy: 0.8624 - val_loss: 0.3629 - val_accuracy: 0.8455
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3334 - accuracy: 0.8630 - val_loss: 0.3656 - val_accuracy: 0.84

In [36]:
model.save('model.h5')

c:\Users\Lenovo\Desktop\udemy\ANN-Clasification\ann\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
## Load Tensorboard Extension
%load_ext tensorboard

In [39]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2336), started 0:02:13 ago. (Use '!kill 2336' to kill it.)